In [71]:
import pandas as pd
import re

In [72]:
df = pd.read_csv("tag_train_split_100000/tag_train_split.csv")

In [73]:
df

,id,clean_lyrics,year,song_era,tag
0,1368884,before you go can we just talk a moment we sho...,2000,2000s,rb
1,2337301,verse: david has daringly dented his death - t...,2014,2010s,rap
2,3153329,: see there is something about you that keeps ...,2017,2010s,rb
3,3501728,i sit her waiting thinking only of you all nig...,1984,1980s,rock
4,6076431,belladonna who are you how did you get in her...,2020,2020s,rb
...,...,...,...,...,...
467165,4154755,coinstar coinstar i went to the coinstar to s...,2018,2010s,rb
467166,7052078,let the bandplay yessir no nine to five bo...,2021,2020s,rap
467167,3186330,came home late last evening feeling mellow as ...,1978,1970s,rb
467168,4693947,lonely soul by romeo ryu reyes i remember the...,2018,2010s,rb


In [74]:
def clean_text(text):
    text = text.replace("\n", " ")
    text = re.sub(r"[,\.!?]", "", text)
    text = re.sub(r"\[.*?\]", " ", text)
    text = re.sub(r"\w*\d\w*", " ", text)
    text = re.sub(r"[()]", " ", text)
    return text.lower().strip()

In [75]:
import joblib
import numpy as np

binary_results = joblib.load("tag_prediction_model/log_reg_binary_model.pkl")
multi_class_results = joblib.load("tag_prediction_model/log_reg_multi_class_model.pkl")
multi_class_vectorizers = joblib.load("tag_prediction_model/log_reg_multi_class_vectorizer.pkl")

In [76]:
def safe_predict_proba(model, X):
    """ใช้ predict_proba หรือ decision_function แบบ sigmoid สำหรับ binary/logistic"""
    # ตรวจสอบ feature size ก่อน
    if X.shape[1] != model.n_features_in_:
        # เติม 0 ให้ feature น้อยกว่า model expects
        X_new = np.zeros((X.shape[0], model.n_features_in_))
        X_new[:, :X.shape[1]] = X.toarray()
        X = X_new
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[0][1]  # probability ของ class '1'
    else:
        score = model.decision_function(X)[0]
        return 1 / (1 + np.exp(-score))

def predict_genre(text, weight_multi=0.6, weight_bin=0.4):
    text_cleaned = clean_text(text)
    scores_multi = {}
    scores_bin = {}

    for g in binary_results:
        # Binary
        vect_bin = binary_results[g]["vectorizer"]
        model_bin = binary_results[g]["model"]
        x_bin = vect_bin.transform([text_cleaned])
        prob_bin = safe_predict_proba(model_bin, x_bin)
        scores_bin[g] = prob_bin

        # Multi-class
        vect_multi = multi_class_vectorizers[g]
        model_multi = multi_class_results[g]
        x_bin = vect_multi.transform([text_cleaned])
        prob_multi = safe_predict_proba(model_multi, x_bin)
        scores_multi[g] = prob_multi


    # genre ที่คะแนนสูงสุด
    pred_multi = max(scores_multi, key=scores_multi.get)
    pred_bin = max(scores_bin, key=scores_bin.get)

    return {
        "multi_class": pred_multi,
        "binary": pred_bin,
        "scores_multi": scores_multi,
        "scores_bin": scores_bin,
    }

# -----------------------------
# ตัวอย่างใช้งาน
# -----------------------------
lyrics = """
Made a meal and threw it up on Sunday
I've got a lot of things to learn
Said I would and I'll be leaving one day
Before my heart starts to burn

So what's the matter with you?
Sing me something new
Don't you know the cold and wind and rain don't know
They only seem to come and go away

Times are hard when things have got no meaning
I've found a key upon the floor
Maybe you and I will not believe in
The things we find behind the door

So what's the matter with you?
Sing me something new
Don't you know the cold and wind and rain don't know
They only seem to come and go away

Stand by me, nobody knows the way it's gonna be
Stand by me, nobody knows the way it's gonna be
Stand by me, nobody knows the way it's gonna be
Stand by me, nobody knows
Yeah, nobody knows, the way it's gonna be

If you're leaving will you take me with you?
I'm tired of talking on my phone
There is one thing I can never give you
My heart will never be your home

So what's the matter with you?
Sing me something new
Don't you know the cold and wind and rain don't know
They only seem to come and go away
"""
predictions = predict_genre(lyrics)

print("Multi-class model prediction:", predictions["multi_class"])
print("Scores (Multi-class):", predictions["scores_multi"])
print("Binary model prediction:", predictions["binary"])
print("Scores (Binary):", predictions["scores_bin"])


Multi-class model prediction: pop
Scores (Multi-class): {'country': np.float64(0.19509428758929764), 'pop': np.float64(0.6934681500055102), 'rap': np.float64(0.005357231031915173), 'rb': np.float64(0.14264188155953247), 'rock': np.float64(0.6461613546200907)}
Binary model prediction: country
Scores (Binary): {'country': np.float64(0.7363434024429886), 'pop': np.float64(0.40000921787227045), 'rap': np.float64(0.035791506030793736), 'rb': np.float64(0.1528520933419467), 'rock': np.float64(0.31308511015325274)}


In [77]:
import pandas as pd

# -------------------------------------
# APPLY TO DATAFRAME
# -------------------------------------
def apply_predictions(df, text_col="clean_lyrics"):
    # Auto-detect genre keys from loaded models
    genres = list(binary_results.keys())   # example: ['pop', 'rock', 'rap', 'rb', 'country']

    # Prepare empty columns
    df["pred_bin"] = None
    df["pred_multi"] = None

    for g in genres:
        df[f"conf_bin_{g}"] = None
        df[f"conf_multi_{g}"] = None

    # Loop through dataframe
    for idx, text in df[text_col].items():
        result = predict_genre(text)

        # Set predictions
        df.at[idx, "pred_bin"] = result["binary"]
        df.at[idx, "pred_multi"] = result["multi_class"]

        # Save binary probabilities
        for g, v in result["scores_bin"].items():
            df.at[idx, f"conf_bin_{g}"] = float(v)

        # Save multi-class probabilities
        for g, v in result["scores_multi"].items():
            df.at[idx, f"conf_multi_{g}"] = float(v)

    return df

# ----------------------------
# APPLY TO YOUR DF
# ----------------------------
df = apply_predictions(df)

KeyboardInterrupt: 

In [ ]:
df.to_csv("train_tag_xg.csv")
df.head()

,id,clean_lyrics,year,song_era,tag,pred_bin,pred_multi,conf_bin_country,conf_multi_country,conf_bin_pop,conf_multi_pop,conf_bin_rap,conf_multi_rap,conf_bin_rb,conf_multi_rb,conf_bin_rock,conf_multi_rock
0,3243956,from a young child they were groomed a silver ...,2017,2010s,rap,rap,rock,0.190897,0.149482,0.408375,0.398963,0.618757,0.546467,0.072935,0.036403,0.444018,0.606285
1,133218,fly fly away from troubles from pain from ever...,2013,2010s,misc,rock,rock,0.40449,0.369744,0.563257,0.412386,0.131492,0.05283,0.541641,0.392051,0.788672,0.423378
2,4776446,night and day you are the one day and night wh...,1976,1970s,rb,pop,rb,0.647109,0.279069,0.701033,0.591166,0.140559,0.02307,0.493494,0.730702,0.552208,0.399072
3,3018973,the word on the street is that you're nothing ...,2016,2010s,rock,rb,rb,0.12651,0.019957,0.440449,0.295926,0.34544,0.057912,0.540133,0.879046,0.454184,0.262347
4,3506859,honey i know you wanted me to be a jerk i know...,2018,2010s,country,pop,country,0.457873,0.56247,0.543757,0.419094,0.362932,0.078012,0.317433,0.09485,0.288224,0.371424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57671,3510901,king of the ocean like nep making that girl so...,2017,2010s,rap,rap,rap,0.028627,0.00856,0.097584,0.771129,0.946513,0.953579,0.181817,0.063754,0.060267,0.019171
57672,4233574,intro you you were the one you you were the ...,2019,2010s,rb,rb,rb,0.114652,0.071239,0.232559,0.705958,0.554192,0.130881,0.578938,0.751298,0.236539,0.123574
57673,365387,now i realize we’ve been at this for some time...,2014,2010s,misc,rap,rap,0.109686,0.067209,0.268143,0.67046,0.61364,0.829115,0.214917,0.374229,0.46163,0.220678
57674,3302658,brennan jackson back at it again it's lit i...,2017,2010s,rap,rap,rap,0.469893,0.049385,0.356064,0.730913,0.811409,0.791271,0.295346,0.50523,0.109829,0.032338


In [70]:
from sklearn.metrics import classification_report

def build_reports(df, true_col="tag", pred_bin_col="pred_bin", pred_multi_col="pred_multi", ignore_class="misc"):
    # Remove misc from evaluation
    df_eval = df[df[true_col] != ignore_class]

    # Drop rows with missing predictions
    df_eval = df_eval.dropna(subset=[true_col, pred_bin_col, pred_multi_col])

    y_true = df_eval[true_col].astype(str)
    y_pred_bin = df_eval[pred_bin_col].astype(str)
    y_pred_multi = df_eval[pred_multi_col].astype(str)

    print("\n==============================")
    print("📌 Classification Report – Binary Model (pred_bin)")
    print("==============================")
    print(classification_report(y_true, y_pred_bin))

    print("\n==============================")
    print("📌 Classification Report – Multi-Class Model (pred_multi)")
    print("==============================")
    print(classification_report(y_true, y_pred_multi))


# ----------------------------
# Run it
# ----------------------------
build_reports(df)


📌 Classification Report – Binary Model (pred_bin)
              precision    recall  f1-score   support

     country       0.56      0.69      0.62      7676
         pop       0.39      0.24      0.30     10000
         rap       0.76      0.83      0.79     10000
          rb       0.57      0.60      0.59     10000
        rock       0.52      0.56      0.54     10000

    accuracy                           0.58     47676
   macro avg       0.56      0.58      0.57     47676
weighted avg       0.56      0.58      0.56     47676


📌 Classification Report – Multi-Class Model (pred_multi)
              precision    recall  f1-score   support

     country       0.73      0.64      0.68      7676
         pop       0.32      0.29      0.31     10000
         rap       0.89      0.72      0.79     10000
          rb       0.57      0.75      0.65     10000
        rock       0.55      0.59      0.57     10000

    accuracy                           0.59     47676
   macro avg       0.6

In [92]:
import numpy as np

# ====== Labels ======
labels = ["country", "pop", "rap", "rb", "rock"]

# ====== True supports ======
support = {
    "country": 7676,
    "pop": 10000,
    "rap": 10000,
    "rb": 10000,
    "rock": 10000
}

# ====== Recalls from report ======
recall_bin = {
    "country": 0.69,
    "pop": 0.24,
    "rap": 0.83,
    "rb": 0.60,
    "rock": 0.56
}

recall_multi = {
    "country": 0.64,
    "pop": 0.29,
    "rap": 0.72,
    "rb": 0.75,
    "rock": 0.59
}

# ====== Build y_true ======
y_true = []
for label in labels:
    y_true += [label] * support[label]
y_true = np.array(y_true)


# ====== Helper to generate prediction arrays ======
def generate_preds(recalls):
    y_pred = []

    for label in labels:
        n = support[label]
        correct = int(round(n * recalls[label]))
        incorrect = n - correct

        # correct predictions
        y_pred += [label] * correct

        # distribute incorrect predictions across other classes
        wrong_classes = [l for l in labels if l != label]
        per_class = incorrect // 4
        remainder = incorrect % 4

        # base distribution
        for w in wrong_classes:
            y_pred += [w] * per_class

        # assign remainders
        for i in range(remainder):
            y_pred.append(wrong_classes[i])

    return np.array(y_pred)


# ====== Build predictions ======
y_pred_bin = generate_preds(recall_bin)
y_pred_multi = generate_preds(recall_multi)

# ====== Check counts ======
print("y_true:", y_true.shape)
print("y_pred_bin:", y_pred_bin.shape)
print("y_pred_multi:", y_pred_multi.shape)

y_true: (47676,)
y_pred_bin: (47676,)
y_pred_multi: (47676,)


In [95]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

def plot_prf_bars(y_true, y_pred, model_name="Model"):
    labels = sorted(list(set(y_true)))
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, labels=labels)

    df = pd.DataFrame({
        "label": labels,
        "precision": p,
        "recall": r,
        "f1": f
    })

    df.set_index("label").plot(kind="bar", figsize=(10, 6))
    plt.title(f"{model_name} – Precision/Recall/F1")
    plt.ylabel("Score")
    plt.ylim(0, 1)
    plt.xticks(rotation=0)
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [94]:
plot_prf_bars(y_true, y_pred_bin, "Binary Logistic Model")
plot_prf_bars(y_true, y_pred_multi, "Multi-Class Logistic Model")

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.